In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder \
  .appName('clean_products') \
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()

In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [ ]:
####################################################### 1 #######################################################

In [27]:
table = "becade_rgarciaf.stg_products"
products = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()

products.printSchema()

root
 |-- app_sale_price: string (nullable = true)
 |-- app_sale_price_currency: string (nullable = true)
 |-- country: string (nullable = true)
 |-- evaluate_rate: string (nullable = true)
 |-- isbestseller: boolean (nullable = true)
 |-- isprime: boolean (nullable = true)
 |-- original_price: string (nullable = true)
 |-- product_detail_url: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_main_image_url: string (nullable = true)
 |-- product_title: string (nullable = true)



In [15]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

In [28]:
products_2=products.toPandas()

In [30]:
	def app_price_normlizer(data):
		for i in data.index:
			data['app_sale_price'][i]=filter(lambda ch: ch in '0123456789,.', data['app_sale_price'][i])
			data['app_sale_price'][i]=''.join(list(data['app_sale_price'][i]))
		
			if(data['app_sale_price'][i].find(',')!=-1):
				data['app_sale_price'][i]=data['app_sale_price'][i].replace(',','.')

				if(data['app_sale_price'][i].count('.')>1):
					if(data['app_sale_price'][i].count('.')==2):
						data['app_sale_price'][i]=data['app_sale_price'][i].replace('.','',1)

			if( (len(data['app_sale_price'][i])-data['app_sale_price'][i].rfind('.') - 1 ) >2):
				data['app_sale_price'][i]=data['app_sale_price'][i].replace('.','',1)

			if(data['app_sale_price'][i]==''):
				data['app_sale_price'][i]='0.00'

			if(data['app_sale_price'][i]==None):
				data['app_sale_price'][i] ='0.00'


		print('Standardized price nomenclature')
		return data

In [31]:
products_3= app_price_normlizer(products_2)

/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

Standardized price nomenclature


In [8]:
def clean_currency(data):
    for i in data.index:
            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and (data["country"][i]=="US" or data["country"][i]=="AU" or data["country"][i]=="CA")):
                data["app_sale_price_currency"][i]="$"

            if( (data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="BR"):
                data["app_sale_price_currency"][i]="R$"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and (data["country"][i]=="FR" or data["country"][i]=="DE" or data["country"][i]=="IT" or data["country"][i]=="NL" or data["country"][i]=="ES")):
                data["app_sale_price_currency"][i]="€"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="IN"):
                data["app_sale_price_currency"][i]="₹"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="TR"):
                data["app_sale_price_currency"][i]="TL"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="AE"):
                data["app_sale_price_currency"][i]="AED"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="GB"):
                data["app_sale_price_currency"][i]="£"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="JP"):
                data["app_sale_price_currency"][i]="¥"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="SG"):
                data["app_sale_price_currency"][i]="S$"

    print('Added missing currencies')
    return data

In [9]:
products_4= clean_currency(products_3)

/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/an

Added missing currencies


In [10]:
table_tasas = "becade_rgarciaf.stg_tasas_cambio_pais_anual "
tasas = spark.read \
  .format("bigquery") \
  .option("table", table_tasas) \
  .load()

tasas.printSchema()

root
 |-- Alpha_2_code: string (nullable = true)
 |-- Alpha_3_code: string (nullable = true)
 |-- Country_name: string (nullable = true)
 |-- currency_year: long (nullable = true)
 |-- currency: string (nullable = true)
 |-- value: double (nullable = true)



In [11]:
tasas_pd = tasas.toPandas()

In [13]:
def generate_app_price_us(data,tasas):
    products_standard_price = {'isbestseller': [] , 'product_id': [] ,'app_sale_price': [], 'app_sale_price_currency': [],'isprime' : [],'evaluate_rate':[],'country':[],'app_sale_price_us':[]}  
    data_sale_price_us=0
    for i in data.index:
        if(data['country'][i]=="US"):
            data_sale_price_us= (float(data['app_sale_price'][i])*1)

        if(data['country'][i]=="AU"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Australia'))]['value']))

        if(data['country'][i]=="CA"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Canada'))]['value']))

        if(data['country'][i]=="BR"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Brazil'))]['value']))

        if(data['country'][i]=="FR" or data['country'][i]=="DE" or data['country'][i]=="IT" or data['country'][i]=="NL" or data['country'][i]=="ES" ):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Germany') & (tasas_pd['currency'] == 'EUR'))]['value']))

        if(data['country'][i]=="IN"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'India'))]['value']))

        if(data['country'][i]=="TR"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Turkey'))]['value']))

        if(data['country'][i]=="AE"):
            data_sale_price_us= (float(data['app_sale_price'][i])/0.27)

        if(data['country'][i]=="GB"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'United Kingdom'))]['value']))

        if(data['country'][i]=="JP"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Japan'))]['value']))

        if(data['country'][i]=="SG"):
            data_sale_price_us= (float(data['app_sale_price'][i])*0.74)
            
        products_standard_price['isbestseller'].append(data['isbestseller'][i])
        products_standard_price['product_id'].append(data['product_id'][i])
        products_standard_price['app_sale_price'].append(data['app_sale_price'][i])
        products_standard_price['app_sale_price_currency'].append(data['app_sale_price_currency'][i])
        products_standard_price['isprime'].append(data['isprime'][i])
        products_standard_price['evaluate_rate'].append(data['evaluate_rate'][i])
        products_standard_price['country'].append(data['country'][i])
        products_standard_price['app_sale_price_us'].append(data_sale_price_us)

    return products_standard_price



In [14]:
products_standard_price = pd.DataFrame(generate_app_price_us(products_4,tasas_pd))

In [89]:
products_standard_price_spark=spark.createDataFrame(products_standard_price) 

root
 |-- isbestseller: boolean (nullable = true)
 |-- product_id: string (nullable = true)
 |-- app_sale_price: string (nullable = true)
 |-- app_sale_price_currency: string (nullable = true)
 |-- isprime: boolean (nullable = true)
 |-- evaluate_rate: string (nullable = true)
 |-- country: string (nullable = true)
 |-- app_sale_price_us: double (nullable = true)

+------------+----------+--------------+-----------------------+-------+-------------+-------+------------------+
|isbestseller|product_id|app_sale_price|app_sale_price_currency|isprime|evaluate_rate|country| app_sale_price_us|
+------------+----------+--------------+-----------------------+-------+-------------+-------+------------------+
|        true|B00PMMBBSG|         14.00|                    AED|  false|         null|     AE| 51.85185185185185|
|        true|B00TU82BC8|        199.56|                    AED|   true|         null|     AE| 739.1111111111111|
|       false|B07NBWPP5W|        346.22|                    AED

In [17]:
products_standard_price_spark_2=products_standard_price_spark.withColumn("app_sale_price",products_standard_price_spark.app_sale_price.cast("decimal(10,2)")) \
    .withColumn("app_sale_price_us",products_standard_price_spark.app_sale_price_us.cast("decimal(10,2)"))

In [18]:
products_standard_price_spark_2.show(10)

+------------+----------+--------------+-----------------------+-------+-------------+-------+-----------------+
|isbestseller|product_id|app_sale_price|app_sale_price_currency|isprime|evaluate_rate|country|app_sale_price_us|
+------------+----------+--------------+-----------------------+-------+-------------+-------+-----------------+
|        true|B00PMMBBSG|         14.00|                    AED|  false|         null|     AE|            51.85|
|        true|B00TU82BC8|        199.56|                    AED|   true|         null|     AE|           739.11|
|       false|B07NBWPP5W|        346.22|                    AED|   true|         null|     AE|          1282.30|
|       false|B08PZ1S496|         40.29|                    AED|   true|         null|     AE|           149.22|
|        true|B097QDYJRZ|         46.99|                    AED|  false|         null|     AE|           174.04|
|       false|B097QDYJRZ|         46.99|                    AED|  false|         null|     AE|  

In [ ]:
products_standard_price_spark_2.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.products_standard_price") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [ ]:
####################################################### 2 #######################################################

In [34]:
from pyspark.sql.functions import sum, col, desc, avg, count

In [24]:
products_avg_price = products_standard_price_spark_2.groupBy("product_id").agg(avg("app_sale_price_us").alias("avg_price_us"),countDistinct("country").alias("count_country")) 

AssertionError: all exprs should be Column

In [ ]:
products_avg_price.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.products_avg_price") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [ ]:
####################################################### 3 #######################################################

In [65]:
products_id=products_standard_price_spark_2.select("product_id").distinct()

In [69]:
products_id_pd=products_id.toPandas()
products_standard_price_pd = products_standard_price_spark_2.toPandas()

In [76]:
def find_max_and_min(data,product_id):
		products_price_ranges = {'product_id': [] ,'max_price': [], 'max_country': [],'min_price' : [],'min_country':[]}  
		max_price=0
		max_country=''
		min_price=10000000000
		min_country=''
		for product in product_id['product_id']:
			for i in data.index:
				if (data["product_id"][i]==product):
					if max_price < data["app_sale_price_us"][i]:
						max_price=data["app_sale_price_us"][i]
						max_country = data["country"][i]

					if min_price > data["app_sale_price_us"][i]:
						min_price=data["app_sale_price_us"][i]
						min_country = data["country"][i]

			products_price_ranges['product_id'].append(product)
			products_price_ranges['max_price'].append(max_price)
			products_price_ranges['max_country'].append(max_country)
			products_price_ranges['min_price'].append(min_price)
			products_price_ranges['min_country'].append(min_country)
			max_price=0
			max_country=''
			min_price=10000000000
			min_country=''

		return products_price_ranges

In [ ]:
products_price_ranges = pd.DataFrame(find_max_and_min(products_standard_price,products_id_pd))

In [92]:
products_price_range_spark=spark.createDataFrame(products_price_ranges) 
products_price_range_spark.printSchema()
products_price_range_spark.show()

root
 |-- product_id: string (nullable = true)
 |-- max_price: double (nullable = true)
 |-- max_country: string (nullable = true)
 |-- min_price: double (nullable = true)
 |-- min_country: string (nullable = true)

+----------+------------------+-----------+------------------+-----------+
|product_id|         max_price|max_country|         min_price|min_country|
+----------+------------------+-----------+------------------+-----------+
|B08KTV3P9M|               0.0|           |               0.0|         SG|
|B0825K4GTM|58.537578208010004|         IT|58.537578208010004|         IT|
|B07C29DXX1|24.568649897644786|         IT|24.568649897644786|         IT|
|B07SJ14XN3| 98.70513182561086|         GB| 87.93771992653053|         IT|
|B07VVPX7YL| 29.68569088051083|         DE| 29.68569088051083|         DE|
|B07TMNKTVJ| 75.62820790223127|         GB|46.099056711712855|         NL|
|B07MSN639J| 65.73341709016537|         DE| 65.73341709016537|         DE|
|B07PCXZ14V|  74.2313601509965|   

In [95]:
products_price_range_spark_2=products_price_range_spark.withColumn("max_price",products_price_range_spark.max_price.cast("decimal(10,2)")) \
    .withColumn("min_price",products_price_range_spark.min_price.cast("decimal(10,2)"))
products_price_range_spark_2.show()

+----------+---------+-----------+---------+-----------+
|product_id|max_price|max_country|min_price|min_country|
+----------+---------+-----------+---------+-----------+
|B08KTV3P9M|     0.00|           |     0.00|         SG|
|B0825K4GTM|    58.54|         IT|    58.54|         IT|
|B07C29DXX1|    24.57|         IT|    24.57|         IT|
|B07SJ14XN3|    98.71|         GB|    87.94|         IT|
|B07VVPX7YL|    29.69|         DE|    29.69|         DE|
|B07TMNKTVJ|    75.63|         GB|    46.10|         NL|
|B07MSN639J|    65.73|         DE|    65.73|         DE|
|B07PCXZ14V|    74.23|         DE|    55.04|         CA|
|B074DDQ629|    13.88|         GB|    13.88|         GB|
|B07SQZWVWP|    17.12|         ES|    17.12|         ES|
|B07H15C55D|    12.81|         GB|    12.81|         GB|
|B08GJTPS2D|     6.05|         IN|     6.05|         IN|
|B085S9XSWG|     9.86|         GB|     9.86|         GB|
|B07TLYTDXF|    19.01|         CA|    19.01|         CA|
|B01N5RJW49|    15.99|         

In [96]:
products_price_range_spark_2.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.products_price_range") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [ ]:
####################################################### 4 #######################################################

In [47]:
table_standar_price = "becade_rgarciaf.products_standard_price "
standar_price = spark.read \
  .format("bigquery") \
  .option("table", table_standar_price) \
  .load()

standar_price.printSchema()

root
 |-- isbestseller: boolean (nullable = true)
 |-- product_id: string (nullable = true)
 |-- app_sale_price: decimal(38,9) (nullable = true)
 |-- app_sale_price_currency: string (nullable = true)
 |-- isprime: boolean (nullable = true)
 |-- evaluate_rate: string (nullable = true)
 |-- country: string (nullable = true)
 |-- app_sale_price_us: decimal(38,9) (nullable = true)



In [39]:
standar_price.show()

+------------+----------+--------------+-----------------------+-------+------------------+-------+-----------------+
|isbestseller|product_id|app_sale_price|app_sale_price_currency|isprime|     evaluate_rate|country|app_sale_price_us|
+------------+----------+--------------+-----------------------+-------+------------------+-------+-----------------+
|       false|B002VPE1QG|  18.750000000|                     S$|  false|4.5 out of 5 stars|     SG|     13.880000000|
|       false|B00EEBS25Q|  53.200000000|                     S$|  false|4.5 out of 5 stars|     SG|     39.370000000|
|       false|B014RWWOAA|  17.920000000|                     S$|  false|4.5 out of 5 stars|     SG|     13.260000000|
|       false|B01GGKYAIU|          0E-9|                     S$|  false|4.5 out of 5 stars|     SG|             0E-9|
|       false|B01MAWUEUI|  22.250000000|                     S$|  false|4.5 out of 5 stars|     SG|     16.470000000|
|       false|B06XF6P1CG|  57.270000000|                

In [48]:
standar_price_pd=standar_price.toPandas()

In [25]:
standar_price_pd.dtypes

isbestseller                 bool
product_id                 object
app_sale_price             object
app_sale_price_currency    object
isprime                      bool
evaluate_rate              object
country                    object
app_sale_price_us          object
dtype: object

In [49]:
print(standar_price_pd)

      isbestseller  product_id app_sale_price app_sale_price_currency  \
0            False  B002VPE1QG   18.750000000                      S$   
1            False  B00EEBS25Q   53.200000000                      S$   
2            False  B014RWWOAA   17.920000000                      S$   
3            False  B01GGKYAIU           0E-9                      S$   
4            False  B01MAWUEUI   22.250000000                      S$   
...            ...         ...            ...                     ...   
5347          True  B0835DZSMT   68.850000000                       $   
5348          True  B083BWG5X1   66.900000000                       $   
5349          True  B00CD8ADKO  699.000000000                       ₹   
5350          True  B00IHMFIBY  539.000000000                       ₹   
5351          True  B0134QJH4G  949.000000000                       ₹   

      isprime       evaluate_rate country app_sale_price_us  
0       False  4.5 out of 5 stars      SG      13.880000000  

In [50]:
def filter_evaluate_range(data):
    for i in data.index:
        if(data["evaluate_rate"][i]=="" or data["evaluate_rate"][i]==None):
            data["evaluate_rate"][i]="0.0"
        data["evaluate_rate"][i] = ''.join((filter(lambda x: x in '0123456789,.', data["evaluate_rate"][i])))
        if(data["evaluate_rate"][i].count('5') >1 ):
            if( data["evaluate_rate"][i][data["evaluate_rate"][i].rfind('5')-1] == ',' or '.' ) :
                list1= list(data["evaluate_rate"][i])
                list1[data["evaluate_rate"][i].find('5')]=""
                data["evaluate_rate"][i]=''.join(list1)
            if(len(data["evaluate_rate"][i]) > (data["evaluate_rate"][i].find('5')+1) ):	
                if (data["evaluate_rate"][i][data["evaluate_rate"][i].find('5')+1] ==  ',' or '.' ) :
                    list1 = list(data["evaluate_rate"][i])
                    list1[data["evaluate_rate"][i].rfind('5')]=""
                    data["evaluate_rate"][i]=''.join(list1)
        else:
            data["evaluate_rate"][i] = ''.join((filter(lambda x: x not in '5', data["evaluate_rate"][i])))
        data["evaluate_rate"][i]=data["evaluate_rate"][i].replace(",",".")

    return data


In [53]:
print(standar_price_pd)

      isbestseller  product_id app_sale_price app_sale_price_currency  \
0            False  B002VPE1QG   18.750000000                      S$   
1            False  B00EEBS25Q   53.200000000                      S$   
2            False  B014RWWOAA   17.920000000                      S$   
3            False  B01GGKYAIU           0E-9                      S$   
4            False  B01MAWUEUI   22.250000000                      S$   
...            ...         ...            ...                     ...   
5347          True  B0835DZSMT   68.850000000                       $   
5348          True  B083BWG5X1   66.900000000                       $   
5349          True  B00CD8ADKO  699.000000000                       ₹   
5350          True  B00IHMFIBY  539.000000000                       ₹   
5351          True  B0134QJH4G  949.000000000                       ₹   

      isprime evaluate_rate country app_sale_price_us  
0       False           4.5      SG      13.880000000  
1       Fal

In [52]:
standar_price_pd_filtered= filter_evaluate_range(standar_price_pd)

/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/anaconda/lib/py

In [54]:
print(standar_price_pd_filtered)

      isbestseller  product_id app_sale_price app_sale_price_currency  \
0            False  B002VPE1QG   18.750000000                      S$   
1            False  B00EEBS25Q   53.200000000                      S$   
2            False  B014RWWOAA   17.920000000                      S$   
3            False  B01GGKYAIU           0E-9                      S$   
4            False  B01MAWUEUI   22.250000000                      S$   
...            ...         ...            ...                     ...   
5347          True  B0835DZSMT   68.850000000                       $   
5348          True  B083BWG5X1   66.900000000                       $   
5349          True  B00CD8ADKO  699.000000000                       ₹   
5350          True  B00IHMFIBY  539.000000000                       ₹   
5351          True  B0134QJH4G  949.000000000                       ₹   

      isprime evaluate_rate country app_sale_price_us  
0       False           4.5      SG      13.880000000  
1       Fal

In [55]:
standar_price_spark_filtered=spark.createDataFrame(standar_price_pd_filtered) 
standar_price_spark_filtered.printSchema()
standar_price_spark_filtered.show()

root
 |-- isbestseller: boolean (nullable = true)
 |-- product_id: string (nullable = true)
 |-- app_sale_price: decimal(38,18) (nullable = true)
 |-- app_sale_price_currency: string (nullable = true)
 |-- isprime: boolean (nullable = true)
 |-- evaluate_rate: string (nullable = true)
 |-- country: string (nullable = true)
 |-- app_sale_price_us: decimal(38,18) (nullable = true)

+------------+----------+--------------------+-----------------------+-------+-------------+-------+--------------------+
|isbestseller|product_id|      app_sale_price|app_sale_price_currency|isprime|evaluate_rate|country|   app_sale_price_us|
+------------+----------+--------------------+-----------------------+-------+-------------+-------+--------------------+
|       false|B002VPE1QG|18.75000000000000...|                     S$|  false|          4.5|     SG|13.88000000000000...|
|       false|B00EEBS25Q|53.20000000000000...|                     S$|  false|          4.5|     SG|39.37000000000000...|
|      

In [61]:
from pyspark.sql.functions import sum, col, desc, avg, count

In [68]:
product_rate_avg=standar_price_spark_filtered.groupBy("product_id")\
    .agg(avg("evaluate_rate").alias("avg_evaluation_rate"),count("country").alias("country_count"))

In [69]:
print(product_rate_avg)

+----------+-------------------+-------------+
|product_id|avg_evaluation_rate|country_count|
+----------+-------------------+-------------+
|B085S9XSWG|                4.5|            1|
|B07TLYTDXF|                4.7|            2|
|B07PCXZ14V|  4.633333333333333|            3|
|B073Q88F7K|                4.3|            1|
|B07SQZWVWP|                4.5|            7|
|B07H15C55D| 4.3999999999999995|            7|
|B01N5RJW49|                4.7|            2|
|B07TMNKTVJ|                4.3|            2|
|B07SJ14XN3|  4.533333333333333|            3|
|B00JRW0GGU|                3.9|            1|
|B08KTV3P9M|                0.0|            1|
|B07C29DXX1|                4.4|            1|
|B07VVPX7YL|                4.1|            2|
|B074DDQ629|                3.7|            1|
|B08GJTPS2D|                4.5|            1|
|B07VXVDYD4|                4.1|            2|
|B0825K4GTM|                4.3|            1|
|B07MSN639J|                4.6|            1|
|B07BN43CMG| 

In [71]:
product_rate_avg_2 = product_rate_avg.withColumn("avg_evaluation_rate",product_rate_avg.avg_evaluation_rate.cast("decimal(10,2)"))

In [72]:
print(product_rate_avg_2)

+----------+-------------------+-------------+
|product_id|avg_evaluation_rate|country_count|
+----------+-------------------+-------------+
|B085S9XSWG|               4.50|            1|
|B07TLYTDXF|               4.70|            2|
|B07PCXZ14V|               4.63|            3|
|B073Q88F7K|               4.30|            1|
|B07SQZWVWP|               4.50|            7|
|B07H15C55D|               4.40|            7|
|B01N5RJW49|               4.70|            2|
|B07TMNKTVJ|               4.30|            2|
|B07SJ14XN3|               4.53|            3|
|B00JRW0GGU|               3.90|            1|
|B08KTV3P9M|               0.00|            1|
|B07C29DXX1|               4.40|            1|
|B07VVPX7YL|               4.10|            2|
|B074DDQ629|               3.70|            1|
|B08GJTPS2D|               4.50|            1|
|B07VXVDYD4|               4.10|            2|
|B0825K4GTM|               4.30|            1|
|B07MSN639J|               4.60|            1|
|B07BN43CMG| 

In [73]:
product_rate_avg_2.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.product_rate_avg") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()